In [1]:
!pip install playwright
!playwright install --with-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 13.7 MB/s eta 0:00:00
Installing dependencies...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,375 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,753 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http

In [7]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
import requests

# Fix asyncio issue in Google Colab
nest_asyncio.apply()

# ✅ Check if website is reachable
real_url = "https://www.1mg.com/drugs-all-medicines"  # Use a real website

response = requests.get(real_url)
if response.status_code != 200:
    print("❌ ERROR: Website is not accessible. Try another website.")
else:
    async def scrape_medicine_prices():
        async with async_playwright() as p:
            # Step 1: Launch Headless Browser
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()

            # Step 2: Open Target Pharmacy Website
            await page.goto(real_url, timeout=60000)
            await page.wait_for_load_state("networkidle")  # ✅ Ensure JS loads fully

            # Step 3: Extract Medicine Details
            medicines = []
            medicine_cards = await page.query_selector_all('.style__product-card___3PnWx')  # ✅ Check this class

            if not medicine_cards:
                print("❌ No medicines found. Check CSS selectors or website changes.")
            else:
                for medicine in medicine_cards:
                    name_elem = await medicine.query_selector('.style__product-title___3zxNC')
                    price_elem = await medicine.query_selector('.style__discounted-price___3pK0s')

                    medicine_data = {
                        "Name": await name_elem.inner_text() if name_elem else "N/A",
                        "Price": await price_elem.inner_text() if price_elem else "N/A"
                    }

                    print(medicine_data)  # ✅ Print to debug
                    medicines.append(medicine_data)

                # Step 4: Save Data to CSV
                df = pd.DataFrame(medicines)
                df.to_csv("medicine_prices.csv", index=False)
                print("✅ Scraping Completed. Data saved to 'medicine_prices.csv'.")

            # Step 5: Close Browser
            await browser.close()

    # Step 6: Run Async Function in Google Colab
    asyncio.get_event_loop().run_until_complete(scrape_medicine_prices())

    # Step 7: Download CSV in Google Colab
    from google.colab import files
    files.download("medicine_prices.csv")


❌ No medicines found. Check CSS selectors or website changes.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>